In [ ]:
from datascience import *
import numpy as np

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In this lecture, I am going to use more interactive plots (they look better) so I am using the plotly.express library.  We won't test you on this but it's good to know.

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Lecture 31

In this lecture, we will explore the use of optimization to find the "best" model and derive least-squares regression.

## Slope & Intercept

In [ ]:
def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    """Computes the correlation between columns x and y"""
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)


In [ ]:
def compute_slope(t, x, y):
    """Computes the slope of the regression line"""
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

def compute_intercept(t, x, y):
    """Computes the intercept of the regression line"""
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - compute_slope(t, x, y)*x_mean

In [ ]:
def predict_linear(t, x, y):
    """Return an array of the regressions estimates at all the x values"""
    a = compute_slope(t, x, y)
    b = compute_intercept(t, x, y)
    pred_y = a * t.column(x) + b
    return pred_y

### Error in Estimation

In [ ]:
demographics = Table.read_table('district_demographics2016.csv')
demographics.show(5)

In [ ]:
px.scatter(demographics.to_df(), 
           x="College%", 
           y="Median Income",
           color="State")

In [ ]:
correlation(demographics, 'College%', 'Median Income')

In [ ]:
slope = compute_slope(demographics, 'College%', 'Median Income')
intercept = compute_intercept(demographics, 'College%', 'Median Income')
print("Slope:", slope)
print("Intercept:", intercept)

In [ ]:
demographics = demographics.with_column(
    "Linear Prediction", predict_linear(demographics, 'College%', 'Median Income')
)

In [ ]:
demographics.iscatter('College%', ["Median Income", "Linear Prediction"])

In [ ]:
fig = px.scatter(demographics.to_df(), x="College%", y="Median Income")
xtest = np.arange(0, 75, 1)
fig.add_scatter(x=xtest, 
                y=slope * xtest + intercept,
                name = f"{np.round(slope, 2)} x + {np.round(intercept)}")
fig

## Computing the Error

In [ ]:
actual = demographics.column('Median Income')
predicted = predict_linear(demographics, 'College%', 'Median Income')

errors = actual - predicted

In [ ]:
demographics = demographics.with_column('Error', errors)
demographics

### Visualizing the Errors

In [ ]:
fig = px.scatter(demographics.to_df(), x="College%", y="Median Income")
xtest = np.arange(0, 75, 1)
fig.add_scatter(x=xtest, 
                y=slope * xtest + intercept,
                name = f"{np.round(slope, 2)} x + {np.round(intercept)}")
fig.add_scatter(x=demographics.column("College%").repeat(3), 
                y=np.ravel(np.vstack([actual, predicted, np.nan * predicted]).T),
                marker_color="gray", line_width=0.75, name="Errors")
fig

In [ ]:
demographics.ihist('Error')

## Summarizing the Overall Error

In [ ]:
np.mean(errors)

Mean Absolute Error

In [ ]:
np.mean(np.abs(errors))

Mean Squared Error (MSE)

In [ ]:
np.mean(errors ** 2)

Root Mean Squared Error (RMSE)

In [ ]:
np.sqrt(np.mean(errors ** 2))

## Error as Function of our Model (Line)

In [ ]:
def demographics_rmse(slope, intercept):
    predicted = slope * demographics.column("College%") + intercept 
    actual = demographics.column("Median Income")
    errors = predicted - actual
    rmse = np.sqrt(np.mean(errors ** 2))
    return rmse

In [ ]:
demographics_rmse(slope, intercept)

What if we used a different slope and intercept value:

In [ ]:
def visualize_demographics_rmse(slope, intercept):
    rmse = demographics_rmse(slope, intercept)
    predicted = slope * demographics.column("College%") + intercept 
    actual = demographics.column("Median Income")
    fig = px.scatter(demographics.to_df(), x="College%", y="Median Income")
    xtest = np.arange(0, 75, 1)
    fig.add_scatter(x=xtest, y=slope * xtest + intercept,
                    name = f"{np.round(slope, 2)} x + {np.round(intercept)}")
    fig.add_scatter(x=demographics.column("College%").repeat(3), 
                    y=np.ravel(np.vstack([actual, predicted, np.nan * predicted]).T),
                    marker_color="gray", line_width=0.75, name="Errors")
    fig.update_layout(title=f"RMSE = {np.round(rmse, 2)}")
    return fig

In [ ]:
visualize_demographics_rmse(slope, intercept)

In [ ]:
visualize_demographics_rmse(slope+1000, intercept - 50000)

Varying the Slope and Intercept and Plotting the RMSE

In [ ]:
alt_slopes = slope + np.arange(-20, 20)
rmses = []
for new_slope in alt_slopes:
    rmses = np.append(rmses, demographics_rmse(new_slope, intercept))

variations = Table().with_columns("Slope", alt_slopes, "RMSE", rmses)
variations

In [ ]:
fig = px.scatter(variations.to_df(), x="Slope", y="RMSE")
fig.add_scatter(x=[slope], y=[demographics_rmse(slope, intercept)], marker_size=10, 
                name="Best Slope")

What if we also tried different intercept terms:

In [ ]:
alt_slopes = slope + np.arange(-100, 100, 1)
alt_intercepts = intercept + np.arange(-2000, 2000, 10)
variations = Table(["Slope", "Intercept", "RMSE"])
for new_slope in alt_slopes:
    for new_intercept in alt_intercepts:
        rmse = demographics_rmse(new_slope, new_intercept)
        variations.append([new_slope, new_intercept, rmse])
    
variations
go.Figure(data=[
    go.Contour(x=variations.column("Slope"), y=variations.column("Intercept"), z=variations.column("RMSE")), 
    go.Scatter(x=[slope], y=[intercept], marker_color="red")
],
layout=dict(width = 800,height=600, xaxis_title="Slope", yaxis_title="Intercept"))

In [ ]:
go.Figure(data=[
    go.Surface(x = alt_slopes, y = alt_intercepts,
               z=variations.column("RMSE").reshape(len(alt_slopes), len(alt_intercepts)).T),
    go.Scatter3d(x=[slope], y=[intercept], z=[demographics_rmse(slope, intercept)])], 
          layout=dict(width = 1200,height=1000, 
                      scene_xaxis_title="Slope", scene_yaxis_title="Intercept", 
                      scene_zaxis_title="RMSE"))

### Numerical Optimization

If our goal is just to find the parameters of our line that minimize some kind of error, we can use numerical optimization tools:

In [ ]:
def f(x):
    return ((x-2)**2) + 3

In [ ]:
x = np.arange(1, 3, 0.1)
y = f(x)
px.line(x=x, y=y)

In [ ]:
minimize(f)

In [ ]:
print("x_min =", minimize(f))
print("f(x_min) =", f(minimize(f)))

In [ ]:
fig = px.line(x=x, y=y)
fig.add_scatter(x=[minimize(f)], y=[f(minimize(f))],
                name="Minimum", marker_color="red", marker_size=10)

Even for more complex functions like:

In [ ]:
def complicated_function(x):
    return 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 

In [ ]:
x = np.arange(-1.5, 1.5, 0.05)
y2 = complicated_function(x)
px.line(x=x, y=y2)

We can still use minimize to find the minimum:

In [ ]:
x_min = minimize(complicated_function)
print("x_min =", x_min)
print("f(x_min) =", complicated_function(x_min))

In [ ]:
fig = px.line(x=x, y=y2)
fig.add_scatter(x=[x_min],
                y=[complicated_function(x_min)],
                name="Minimum", marker_color="red", marker_size=10)

### Minimizing RMSE 

We can use minimize to find the slope and intercept that minimize root mean squared error in our predictions:

In [ ]:
minimize(demographics_rmse)

How does this compare to the slope and intercept we derived earlier?

In [ ]:
[slope, intercept]

What happens if we minimize the mean squared error instead of the root mean squared error?

In [ ]:
def demographics_mse(slope, intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = slope*x + intercept
    return np.mean(((y - estimate) ** 2))

In [ ]:
minimize(demographics_mse)

What if we wanted to use the absolute error instead?

In [ ]:
def demographics_mae(any_slope, any_intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = any_slope*x + any_intercept
    return np.mean(np.abs(y - estimate))

In [ ]:
minimize(demographics_mae)

That is different!

In [ ]:
mae_slope, mae_intercept = minimize(demographics_mae)
fig = px.scatter(demographics.to_df(), x="College%", y="Median Income", color="State")
xtest = np.arange(0, 75, 0.1)
fig.add_scatter(x=xtest, 
                y=slope * xtest + intercept,
                name = f"Least Squares: {np.round(slope, 2)} x + {np.round(intercept)}")
fig.add_scatter(x=xtest, 
                y=mae_slope * xtest + mae_intercept,
                name = f"MAE: {np.round(mae_slope, 2)} x + {np.round(mae_intercept)}")
fig

In [ ]:
demographics_rmse(1500, 20000)

In [ ]:
demographics_rmse(1350, 19000)

In [ ]:
demographics_rmse(-1000, 75000)

In [ ]:
minimize(demographics_rmse)

In [ ]:
make_array(regression_slope, regression_intercept)

### Nonlinear Regression ###

In [ ]:
shotput = Table.read_table('shotput.csv')
shotput

In [ ]:
shotput.scatter('Weight Lifted')

In [ ]:
def shotput_linear_rmse(any_slope, any_intercept):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = any_slope*x + any_intercept
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
best_line = minimize(shotput_linear_rmse)
best_line

In [ ]:
weights = shotput.column(0)

In [ ]:
linear_fit = best_line.item(0)*weights + best_line.item(1)

shotput.with_column(
    'Best Line', linear_fit
).scatter(0)

**Quadratic Function**

$$
f(x) ~=~ ax^2 + bx + c
$$
for constants $a$, $b$, and $c$.



In [ ]:
def shotput_quadratic_rmse(a, b, c):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = a*(x**2) + b*x + c
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
best_quad = minimize(shotput_quadratic_rmse)
best_quad

In [ ]:
# x = weight lifted = 100 kg
# Then predicted shot put distance:

(-0.00104)*(100**2) + 0.2827*100 - 1.5318

In [ ]:
quad_fit = best_quad.item(0)*(weights**2) + best_quad.item(1)*weights + best_quad.item(2)

In [ ]:
shotput.with_column('Best Quadratic Curve', quad_fit).scatter(0)

In [ ]:
def demographics_errors(slope, intercept):
    # Use four convenient points from the original data
    sample = [[14.7, 33995], [19.1, 61454], [50.7, 71183], [59.5, 105918]]
    demographics.scatter('College%', 'Median Income', alpha=0.5)
    xlims = make_array(5, 75)
    # Plot a line with the slope and intercept you specified:
    plots.plot(xlims, slope * xlims + intercept, lw=4)
    # Plot red lines from each of the four points to the line
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=4)

In [ ]:
def show_demographics_rmse(slope, intercept):
    demographics_errors(slope, intercept)
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    prediction = slope * x + intercept
    mse = np.mean((y - prediction) ** 2)
    print("Root mean squared error:", round(mse ** 0.5, 2))